### Combine silver layer tables and create a Report Data Frame

In [0]:
import datetime


reservations_df = spark.read.format('delta').load(f'{SILVER_LAYER_PATH}/reservations')
flights_df = spark.read.format('delta').load(f'{SILVER_LAYER_PATH}/flights')
hotels_df = spark.read.format('delta').load(f'{SILVER_LAYER_PATH}/hotels')
customers_df = spark.read.format('delta').load(f'{SILVER_LAYER_PATH}/customers')

# Join all tables to create final report table
report_df = reservations_df \
    .join(flights_df, reservations_df.flight_id == flights_df.flight_id, how='left') \
    .join(hotels_df, reservations_df.hotel_id == hotels_df.hotel_id, how='left') \
    .join(customers_df, reservations_df.customer_id == customers_df.customer_id, how='left')

# Select final report table columns in order
report_df = report_df.select(
    'reservation_id',
    'reservation_date',
    'customer_gender',
    'customer_age',
    'flight_departure_date',
    'flight_airline_name',
    'flight_plane_model',
    'hotel_stars',
    'room_type',
    'payment_method',
    'total_price_amount',
    'total_price_currency',
    'total_price_amount_in_eur',
)

# Save report table to gold layer
report_df.write \
    .format('delta') \
    .mode('overwrite') \
    .option('mergeSchema', 'true') \
    .save(f'{GOLD_LAYER_PATH}/report_table')

print(f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} | Table "report_table" saved to gold layer.')